Train Data Loading ...

In [1]:
import pandas as pd
from posixpath import sep
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.txt",sep="\t+" , header = None)
dataTest1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.txt",sep="\t+" , header = None)
dataTest2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dev.txt",sep="\t+" , header = None)
column_names = ["ID", "Turn1", "Turn2", "Turn3" , "Label" ]
data.columns = column_names
data['Conversation'] = data['Turn1'] + " " + data["Turn2"] + " " + data["Turn3"]
dataTest1.columns = column_names
dataTest1['Conversation'] = dataTest1['Turn1'] + " " + dataTest1["Turn2"] + " " + dataTest1["Turn3"]
dataTest2.columns = column_names
dataTest2['Conversation'] = dataTest2['Turn1'] + " " + dataTest2["Turn2"] + " " + dataTest2["Turn3"]

<ipython-input-1-fbc3ae9e4ee7>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.txt",sep="\t+" , header = None)
<ipython-input-1-fbc3ae9e4ee7>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataTest1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.txt",sep="\t+" , header = None)
<ipython-input-1-fbc3ae9e4ee7>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specify

**Data Preprocessing  **                                                
**1 Converting to lowercase**

In [2]:
data['Conversation'] = data['Conversation'].str.lower()
data["Label"] = data["Label"].str.lower()
dataTest1['Conversation'] = dataTest1['Conversation'].str.lower()
dataTest1["Label"] = dataTest1["Label"].str.lower()
dataTest2['Conversation'] = dataTest2['Conversation'].str.lower()
dataTest2["Label"] = dataTest2["Label"].str.lower()


**2 Removing Punctuation**

In [3]:
import string
string.punctuation
def remove_punctuation(txt):
  txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
  return txt_nopunct
data["Conversation"] = data["Conversation"].apply(lambda x: remove_punctuation(x))
dataTest1["Conversation"] = dataTest1["Conversation"].apply(lambda x: remove_punctuation(x))
dataTest2["Conversation"] = dataTest2["Conversation"].apply(lambda x: remove_punctuation(x))


**3 Converting Emojis**

In [4]:
!pip install emoji
import emoji

def replace_emojis_with_meanings(text):
    return emoji.demojize(text)

data["Conversation"] = data["Conversation"].apply(replace_emojis_with_meanings)
dataTest1["Conversation"] = dataTest1["Conversation"].apply(replace_emojis_with_meanings)
dataTest2["Conversation"] = dataTest2["Conversation"].apply(replace_emojis_with_meanings)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 6.2 MB/s eta 0:00:00


**4 Replacing Short Forms**

In [5]:
!pip install contractions
import pandas as pd
import contractions

# Function to replace short forms using contractions library
def replace_short_forms(text):
    return contractions.fix(text)

# Apply the function to the 'text' column
data['Conversation'] = data['Conversation'].apply(replace_short_forms)
dataTest1['Conversation'] =  dataTest1['Conversation'].apply(replace_short_forms)
dataTest2['Conversation'] =  dataTest2['Conversation'].apply(replace_short_forms)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 8.8 MB/s eta 0:00:00


**6 Removing digits**

In [6]:
data['Conversation'] = data['Conversation'].replace(to_replace=r'[^\w\s]', value='', regex=True)
dataTest1['Conversation'] = dataTest1['Conversation'].replace(to_replace=r'[^\w\s]', value='', regex=True)
dataTest2['Conversation'] = dataTest2['Conversation'].replace(to_replace=r'[^\w\s]', value='', regex=True)

In [7]:
column_values = data['Conversation'].values
column_values = data['Conversation'].to_numpy()
label_array = data["Label"].values
label_array = data["Label"].to_numpy()
column_values_test1 = dataTest1['Conversation'].values
column_values_test1 = dataTest1['Conversation'].to_numpy()
label_array_test1 = dataTest1["Label"].values
label_array_test1 = dataTest1["Label"].to_numpy()

column_values_test2 = dataTest2['Conversation'].values
column_values_test2 = dataTest2['Conversation'].to_numpy()
label_array_test2 = dataTest2["Label"].values
label_array_test2 = dataTest2["Label"].to_numpy()

In [8]:
# Import Section
import csv
import codecs
import sys
import io
import numpy as np

In [9]:
!pip install ekphrasis
# For HashtagSegmentation
from ekphrasis.classes.segmenter import Segmenter

# For Classifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
# Python script for confusion matrix creation.
from sklearn.metrics import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 4.4 MB/s eta 0:00:00


 **Multinomial Naive Bayes **


In [10]:


def main():
  Conversations = []
  label = []
  l = len(label_array)
  for i in range (1,l) :
    Conversations.append(column_values[i])
    label.append(label_array[i])
    #Conversations.append(data['Conversation'][i])
    #label.append(label_array[i])

  X_train = np.array(Conversations)
  Y_train = np.array(label)


  ########For Multinomial Naive Bayes:########
  from sklearn.svm import LinearSVC
  from sklearn.svm import SVC

  from sklearn.naive_bayes import MultinomialNB
  classifier = Pipeline([
     ('count_vectorizer', CountVectorizer(ngram_range=(1, 3))),
      ('tfidf', TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)),
     ('clf', OneVsRestClassifier(MultinomialNB()))])


  ## Train Classifier
  classifier.fit(X_train, Y_train)

  testTConversations = []
  testLabel = []
  l = len(column_values_test1)
  for i in range (1,l) :
    testTConversations.append(column_values_test1[i])
    testLabel.append(label_array_test1[i])

  l = len(column_values_test2)
  for i in range (1,l) :
    testTConversations.append(column_values_test2[i])
    testLabel.append(label_array_test2[i])

  X_test = np.array(testTConversations)
  testLabelPredicted = classifier.predict(X_test)

  # Evaluation
  results = confusion_matrix(testLabel, testLabelPredicted)

  print ('Confusion Matrix :')
  print (results)

  #print ('Recall Score :',recall_score(testLabel, testLabelPredicted))
  #print ('Precision Score :',precision_score(testLabel, testLabelPredicted))
  print ('Macro Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='macro'))
  print ('Micro Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='micro'))
  print ('Weighted Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='weighted'))
  print ('Accuracy :',accuracy_score(testLabel, testLabelPredicted))

  #print_confusion_matrix(results,["Happy","Sad","Angry","Others"])

  print ('Evaluation Report : ')
  print (classification_report(testLabel, testLabelPredicted))

if __name__ == '__main__':
  main()

Confusion Matrix :
[[ 142    0  302    4]
 [   0   36  390    0]
 [  26    9 6960   20]
 [   5    0  298   72]]
Macro Avg. F1 Score : 0.46152400395463433
Micro Avg. F1 Score : 0.8724588576960309
Weighted Avg. F1 Score : 0.8361324403952834
Accuracy : 0.8724588576960309
Evaluation Report : 
              precision    recall  f1-score   support

       angry       0.82      0.32      0.46       448
       happy       0.80      0.08      0.15       426
      others       0.88      0.99      0.93      7015
         sad       0.75      0.19      0.31       375

    accuracy                           0.87      8264
   macro avg       0.81      0.40      0.46      8264
weighted avg       0.86      0.87      0.84      8264



**Linear SVC (SVM)**

In [11]:
def main():
  Conversations = []
  label = []
  l = len(label_array)
  for i in range (1,l) :
    Conversations.append(column_values[i])
    label.append(label_array[i])
    #Conversations.append(data['Conversation'][i])
    #label.append(label_array[i])

  X_train = np.array(Conversations)
  Y_train = np.array(label)


  ######## For Linear SVC (SVM) ##########
  from sklearn.svm import LinearSVC
  from sklearn.svm import SVC

  classifier = Pipeline([
     ('count_vectorizer', CountVectorizer(ngram_range=(1, 3))),
     ('tfidf', TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)),
     ('clf', OneVsRestClassifier(LinearSVC(C=10.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
      verbose=0)))])


  ## Train Classifier
  classifier.fit(X_train, Y_train)

  testTConversations = []
  testLabel = []
  l = len(column_values_test1)
  for i in range (1,l) :
    testTConversations.append(column_values_test1[i])
    testLabel.append(label_array_test1[i])

  l = len(column_values_test2)
  for i in range (1,l) :
    testTConversations.append(column_values_test2[i])
    testLabel.append(label_array_test2[i])

  X_test = np.array(testTConversations)
  testLabelPredicted = classifier.predict(X_test)

  # Evaluation
  results = confusion_matrix(testLabel, testLabelPredicted)

  print ('Confusion Matrix :')
  print (results)

  #print ('Recall Score :',recall_score(testLabel, testLabelPredicted))
  #print ('Precision Score :',precision_score(testLabel, testLabelPredicted))
  print ('Macro Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='macro'))
  print ('Micro Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='micro'))
  print ('Weighted Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='weighted'))
  print ('Accuracy :',accuracy_score(testLabel, testLabelPredicted))

  #print_confusion_matrix(results,["Happy","Sad","Angry","Others"])

  print ('Evaluation Report : ')
  print (classification_report(testLabel, testLabelPredicted))

if __name__ == '__main__':
  main()

Confusion Matrix :
[[ 379    4   52   13]
 [  12  276  129    9]
 [ 424  335 5970  286]
 [  25   10   71  269]]
Macro Avg. F1 Score : 0.645216630593044
Micro Avg. F1 Score : 0.8342207163601162
Weighted Avg. F1 Score : 0.8503104321471552
Accuracy : 0.8342207163601162
Evaluation Report : 
              precision    recall  f1-score   support

       angry       0.45      0.85      0.59       448
       happy       0.44      0.65      0.53       426
      others       0.96      0.85      0.90      7015
         sad       0.47      0.72      0.57       375

    accuracy                           0.83      8264
   macro avg       0.58      0.77      0.65      8264
weighted avg       0.88      0.83      0.85      8264



**SGD Classifier**

In [13]:
def main():
  Conversations = []
  label = []
  l = len(label_array)
  for i in range (1,l) :
    Conversations.append(column_values[i])
    label.append(label_array[i])
    #Conversations.append(data['Conversation'][i])
    #label.append(label_array[i])

  X_train = np.array(Conversations)
  Y_train = np.array(label)


  ########For SGD Classifier:########
  from sklearn.linear_model import SGDClassifier
  classifier = Pipeline([
     ('count_vectorizer', CountVectorizer(ngram_range=(1, 3))),
     ('clf', OneVsRestClassifier(SGDClassifier(loss="hinge", penalty="l2", max_iter=5)))])


  ## Train Classifier
  classifier.fit(X_train, Y_train)

  testTConversations = []
  testLabel = []
  l = len(column_values_test1)
  for i in range (1,l) :
    testTConversations.append(column_values_test1[i])
    testLabel.append(label_array_test1[i])


  l = len(column_values_test2)
  for i in range (1,l) :
    testTConversations.append(column_values_test2[i])
    testLabel.append(label_array_test2[i])

  X_test = np.array(testTConversations)
  testLabelPredicted = classifier.predict(X_test)

  # Evaluation
  results = confusion_matrix(testLabel, testLabelPredicted)

  print ('Confusion Matrix :')
  print (results)

  #print ('Recall Score :',recall_score(testLabel, testLabelPredicted))
  #print ('Precision Score :',precision_score(testLabel, testLabelPredicted))
  print ('Macro Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='macro'))
  print ('Micro Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='micro'))
  print ('Weighted Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='weighted'))
  print ('Accuracy :',accuracy_score(testLabel, testLabelPredicted))

  #print_confusion_matrix(results,["Happy","Sad","Angry","Others"])

  print ('Evaluation Report : ')
  print (classification_report(testLabel, testLabelPredicted))

if __name__ == '__main__':
  main()

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Confusion Matrix :
[[ 355    7   75   11]
 [  14  258  147    7]
 [ 395  307 6083  230]
 [  27   12   86  250]]
Macro Avg. F1 Score : 0.6410439135804686
Micro Avg. F1 Score : 0.840513068731849
Weighted Avg. F1 Score : 0.8537367715547529
Accuracy : 0.840513068731849
Evaluation Report : 
              precision    recall  f1-score   support

       angry       0.45      0.79      0.57       448
       happy       0.44      0.61      0.51       426
      others       0.95      0.87      0.91      7015
         sad       0.50      0.67      0.57       375

    accuracy                           0.84      8264
   macro avg       0.59      0.73      0.64      8264
weighted avg       0.88      0.84      0.85      8264



**Decision Tree Classifier**

In [15]:
def main():
  Conversations = []
  label = []
  l = len(label_array)
  for i in range (1,l) :
    Conversations.append(column_values[i])
    label.append(label_array[i])
    #Conversations.append(data['Conversation'][i])
    #label.append(label_array[i])

  X_train = np.array(Conversations)
  Y_train = np.array(label)


  ########For Decision Tree Classifier:########
  from sklearn.tree import DecisionTreeClassifier

  classifier = Pipeline([
     ('count_vectorizer', CountVectorizer(ngram_range=(1, 3))),
     ('clf', OneVsRestClassifier(DecisionTreeClassifier(random_state=0)))])


  ## Train Classifier
  classifier.fit(X_train, Y_train)

  testTConversations = []
  testLabel = []
  l = len(column_values_test1)
  for i in range (1,l) :
    testTConversations.append(column_values_test1[i])
    testLabel.append(label_array_test1[i])


  l = len(column_values_test2)
  for i in range (1,l) :
    testTConversations.append(column_values_test2[i])
    testLabel.append(label_array_test2[i])

  X_test = np.array(testTConversations)
  testLabelPredicted = classifier.predict(X_test)

  # Evaluation
  results = confusion_matrix(testLabel, testLabelPredicted)

  print ('Confusion Matrix :')
  print (results)

  #print ('Recall Score :',recall_score(testLabel, testLabelPredicted))
  #print ('Precision Score :',precision_score(testLabel, testLabelPredicted))
  print ('Macro Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='macro'))
  print ('Micro Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='micro'))
  print ('Weighted Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='weighted'))
  print ('Accuracy :',accuracy_score(testLabel, testLabelPredicted))

  #print_confusion_matrix(results,["Happy","Sad","Angry","Others"])

  print ('Evaluation Report : ')
  print (classification_report(testLabel, testLabelPredicted))

if __name__ == '__main__':
  main()

Confusion Matrix :
[[ 284   18   82   64]
 [   4  227  133   62]
 [ 189  255 5570 1001]
 [  12    7   62  294]]
Macro Avg. F1 Score : 0.5715759449796313
Micro Avg. F1 Score : 0.7714181994191674
Weighted Avg. F1 Score : 0.8080168210927035
Accuracy : 0.7714181994191674
Evaluation Report : 
              precision    recall  f1-score   support

       angry       0.58      0.63      0.61       448
       happy       0.45      0.53      0.49       426
      others       0.95      0.79      0.87      7015
         sad       0.21      0.78      0.33       375

    accuracy                           0.77      8264
   macro avg       0.55      0.69      0.57      8264
weighted avg       0.87      0.77      0.81      8264



**OneVsRestClassifier**

In [18]:
def main():
  Conversations = []
  label = []
  l = len(label_array)
  for i in range (1,l) :
    Conversations.append(column_values[i])
    label.append(label_array[i])
    #Conversations.append(data['Conversation'][i])
    #label.append(label_array[i])

  X_train = np.array(Conversations)
  Y_train = np.array(label)


  ########For Random Forest Classifier:########


  from sklearn.ensemble import RandomForestClassifier

  classifier = Pipeline([
     ('count_vectorizer', CountVectorizer(ngram_range=(1, 3))),
     ('clf', OneVsRestClassifier(MultinomialNB()))])


  ## Train Classifier
  classifier.fit(X_train, Y_train)

  testTConversations = []
  testLabel = []
  l = len(column_values_test1)
  for i in range (1,l) :
    testTConversations.append(column_values_test1[i])
    testLabel.append(label_array_test1[i])


  l = len(column_values_test2)
  for i in range (1,l) :
    testTConversations.append(column_values_test2[i])
    testLabel.append(label_array_test2[i])

  X_test = np.array(testTConversations)
  testLabelPredicted = classifier.predict(X_test)

  # Evaluation
  results = confusion_matrix(testLabel, testLabelPredicted)

  print ('Confusion Matrix :')
  print (results)

  #print ('Recall Score :',recall_score(testLabel, testLabelPredicted))
  #print ('Precision Score :',precision_score(testLabel, testLabelPredicted))
  print ('Macro Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='macro'))
  print ('Micro Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='micro'))
  print ('Weighted Avg. F1 Score :',f1_score(testLabel, testLabelPredicted, average='weighted'))
  print ('Accuracy :',accuracy_score(testLabel, testLabelPredicted))

  #print_confusion_matrix(results,["Happy","Sad","Angry","Others"])

  print ('Evaluation Report : ')
  print (classification_report(testLabel, testLabelPredicted))

if __name__ == '__main__':
  main()

Confusion Matrix :
[[ 306    3  132    7]
 [  16  129  269   12]
 [ 337   98 6391  189]
 [  28    3  148  196]]
Macro Avg. F1 Score : 0.5874656683270233
Micro Avg. F1 Score : 0.8497095837366893
Weighted Avg. F1 Score : 0.8497575141964345
Accuracy : 0.8497095837366893
Evaluation Report : 
              precision    recall  f1-score   support

       angry       0.45      0.68      0.54       448
       happy       0.55      0.30      0.39       426
      others       0.92      0.91      0.92      7015
         sad       0.49      0.52      0.50       375

    accuracy                           0.85      8264
   macro avg       0.60      0.60      0.59      8264
weighted avg       0.86      0.85      0.85      8264

